# Notebook #2: K Nearest Neighbors and Normalization
**Name: Nick Peters**

**Due: 9/22 before class**

## Load in the data:
You'll notice that there are a lot of columns here. We're going to work with a subset of these columns, not the entire dataset.

In [1]:
#selecting columns in dataframes
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import the data:
#make sure the path on the line below corresponds to the path where you put your dataset.
spotify_data = pd.read_csv('/content/drive/MyDrive/CS167 Datasets/spotify_top_charts.csv')
spotify_data.head()

,uri,artist_names,track_name,peak_rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,time_signature,duration_ms
0,spotify:track:02MWAaffLxlfxAUY7c5dvx,Glass Animals,Heat Waves,1.0,0.761,0.525,11.0,-6.900,1.0,0.0944,0.4400,0.000007,0.0921,80.870,4.0,238805.0
1,spotify:track:5PjdY0CKGZdEuoNab3yDmX,"The Kid LAROI, Justin Bieber",STAY (with Justin Bieber),1.0,0.591,0.764,1.0,-5.484,1.0,0.0483,0.0383,0.000000,0.1030,169.928,4.0,141806.0
2,spotify:track:3IAfUEeaXRX9s9UdKOJrFI,Anitta,Envolver,3.0,0.812,0.736,4.0,-5.421,NaN,0.0833,0.1520,0.002540,0.0914,91.993,4.0,193806.0
3,spotify:track:1HhNoOuqm1a5MXYEgAFl8o,"Imagine Dragons, JID, Arcane, League of Legends",Enemy (with JID) - from the series Arcane Leag...,3.0,0.728,0.783,11.0,-4.424,0.0,0.2660,0.2370,0.000000,0.4340,77.011,4.0,173381.0
4,spotify:track:4fouWK6XVHhzl78KzQ1UjL,GAYLE,abcdefu,1.0,0.695,0.540,4.0,-5.692,1.0,0.0493,0.2990,0.000000,0.3670,121.932,4.0,168602.0


## Exercise #1: 
**Work with the right subset:**

You're not going to work with the whole data set, just the 'artist_names', 'track_name', 'peak_rank', 'duration_ms' and 2-4 additional features of your choice. To start, make this subset of the original data. 

Display the first five rows of your new subset. Also display the number of rows and columns of your subset.

In [4]:
spotify_copy = spotify_data.copy()
spotify_copy = spotify_copy[['artist_names', 'track_name', 'peak_rank', 'danceability', 'energy', 'loudness', 'duration_ms',]]
print(spotify_copy[0:5])
print()
print(spotify_copy.shape)

                                      artist_names  \
0                                    Glass Animals   
1                     The Kid LAROI, Justin Bieber   
2                                           Anitta   
3  Imagine Dragons, JID, Arcane, League of Legends   
4                                            GAYLE   

                                          track_name  peak_rank  danceability  \
0                                         Heat Waves        1.0         0.761   
1                          STAY (with Justin Bieber)        1.0         0.591   
2                                           Envolver        3.0         0.812   
3  Enemy (with JID) - from the series Arcane Leag...        3.0         0.728   
4                                            abcdefu        1.0         0.695   

   energy  loudness  duration_ms  
0   0.525    -6.900     238805.0  
1   0.764    -5.484     141806.0  
2   0.736    -5.421     193806.0  
3   0.783    -4.424     173381.0  
4   0.540    

## Exercise #2:
**Check for null values**: If you have a null target value ('peak_rank'), you will need to throw that example out. 

Replace each of the null values within your predictors with the average of the column.

*hint: Useful functions here are `isna()`, `any()`, `fillna()`, `notna()` and `dropna()`. If you're not sure what these do, look them up in the [pandas documentation](https://pandas.pydata.org/docs/).*

After you have eliminated null values, use `.isna().any()` to verify that your fields no longer contain any null values (all of the values should be False).

In [5]:
print('Before cleaning:', spotify_copy.shape)
print(spotify_copy.isna().any())
spotify_copy.dropna(subset=['peak_rank'], inplace=True)
spotify_copy['danceability'].fillna(spotify_copy['danceability'].mean() ,inplace=True)
spotify_copy['energy'].fillna(spotify_copy['energy'].mean() ,inplace=True)
spotify_copy['loudness'].fillna(spotify_copy['loudness'].mean() ,inplace=True)
spotify_copy['duration_ms'].fillna(spotify_copy['duration_ms'].mean() ,inplace=True)
print()
print('After cleaning:', spotify_copy.shape)
print(spotify_copy.isna().any())

Before cleaning: (646, 7)
artist_names    False
track_name      False
peak_rank        True
danceability     True
energy           True
loudness         True
duration_ms      True
dtype: bool

After cleaning: (638, 7)
artist_names    False
track_name      False
peak_rank       False
danceability    False
energy          False
loudness        False
duration_ms     False
dtype: bool


## Exercise #3
Write up a k-nearest-neighbors function like the one you made for the iris data set in class. It should be able to make peak rank predictions for new songs. You should also be able to specify what you want to use as k.

You will need to incorporate the predictor features that you have chosen to be a part of your subset in your knn function (but not the artist_names nor track_name).

Pay special attention that I want you to refer to the target variable 'peak_rank' as a value (not a category). Thus, you need to adapt the code to handle a regression.

In [6]:
import numpy as np
def knn (specimen, data, k):
  data_copy = data.copy()
  data_copy['distance to new'] = np.sqrt(
    (specimen['danceability']-data_copy['danceability'])**2
    +(specimen['energy']-data_copy['energy'])**2
    +(specimen['loudness']-data_copy['loudness'])**2
    +(specimen['duration_ms']-data_copy['duration_ms'])**2)
  data_copy.sort_values(['distance to new'], inplace=True)
  return data_copy[0:k]

## Exercise #4
Demonstrate that your function works by using the values from a song you recognize on the list. Pass in k=1 in to the funtion, and your algorithm should predict the exact peak_rank as your example song.

Make up some new values for a hypothetical new song.  Use your knn function to display a predicted peak_rank for the song. In this case, use a k value that is greater than 1.

In [7]:
#run if you want to find a song to input as a confirmed case
spotify_copy

,artist_names,track_name,peak_rank,danceability,energy,loudness,duration_ms
0,Glass Animals,Heat Waves,1.0,0.761,0.525,-6.900,238805.0
1,"The Kid LAROI, Justin Bieber",STAY (with Justin Bieber),1.0,0.591,0.764,-5.484,141806.0
2,Anitta,Envolver,3.0,0.812,0.736,-5.421,193806.0
3,"Imagine Dragons, JID, Arcane, League of Legends",Enemy (with JID) - from the series Arcane Leag...,3.0,0.728,0.783,-4.424,173381.0
4,GAYLE,abcdefu,1.0,0.695,0.540,-5.692,168602.0
...,...,...,...,...,...,...,...
641,SALES,Pope Is a Rockstar,108.0,0.725,0.446,-8.958,187333.0
642,Morgan Wallen,Don't Think Jesus,96.0,0.578,0.431,-7.034,226975.0
643,"Lil Nas X, YoungBoy Never Broke Again",Late To Da Party (F*CK BET) (feat. YoungBoy Ne...,86.0,0.872,0.422,-7.415,180666.0
644,"Eminem, Snoop Dogg",From The D 2 The LBC (with Snoop Dogg),135.0,0.730,0.785,-7.986,215120.0


In [8]:
new_specimen = {}
new_specimen['danceability'] = 0.593
new_specimen['energy'] = 0.503
new_specimen['loudness'] = -6.725
new_specimen['duration_ms'] = 137704.0

print(knn(new_specimen, spotify_copy, 1))

   artist_names                      track_name  peak_rank  danceability  \
46    Lil Nas X  MONTERO (Call Me By Your Name)        1.0         0.593   

    energy  loudness  duration_ms  distance to new  
46   0.503    -6.725     137704.0              0.0  


In [9]:
new_specimen = {}
new_specimen['danceability'] = 0.693
new_specimen['energy'] = 0.599
new_specimen['loudness'] = -7.235
new_specimen['duration_ms'] = 255000.0

print(knn(new_specimen, spotify_copy, 3))

                                          artist_names  \
240  Chris Jedi, Anuel AA, Chencho Corleone, Ñengo ...   
448                                     Kendrick Lamar   
615                                              Drake   

                             track_name  peak_rank  danceability  energy  \
240  La Llevo Al Cielo (Ft. Ñengo Flow)       53.0         0.795   0.845   
448                     United In Grief       11.0         0.529   0.845   
615                     Flight's Booked       49.0         0.593   0.410   

     loudness  duration_ms  distance to new  
240    -3.809     254920.0        80.073768  
448    -8.142     255378.0       378.001204  
615   -13.097     254407.0       593.029012  


## Exercise #5

Make a copy of the data and normalize the training data using Z-score.

*hint: `.copy()` will be useful here.* 

Print out the first 5 lines of the normalized subset.

Predict the peak rank with some input parameters using your k-nearest-neighbors function with both the normalized and non-normalized training data. Compare your results. Use a markup cell to describe and explain the cause of any differences. Detailed responses will receive full credit.

In [10]:
normalized_spotify = spotify_copy.copy()


dance_avg = normalized_spotify['danceability'].mean()
dance_std = normalized_spotify['danceability'].std()

e_avg = normalized_spotify['energy'].mean()
e_std = normalized_spotify['energy'].std()

l_avg = normalized_spotify['loudness'].mean()
l_std = normalized_spotify['loudness'].std()

duration_avg = normalized_spotify['duration_ms'].mean()
duration_std = normalized_spotify['duration_ms'].std()

normalized_spotify['danceability'] = (normalized_spotify['danceability'] - dance_avg)/dance_std
normalized_spotify['energy'] = (normalized_spotify['energy'] - e_avg)/e_std
normalized_spotify['loudness'] = (normalized_spotify['loudness'] - l_avg)/l_std
normalized_spotify['duration_ms'] = (normalized_spotify['duration_ms'] - duration_avg)/duration_std

normalized_spotify[0:5]

,artist_names,track_name,peak_rank,danceability,energy,loudness,duration_ms
0,Glass Animals,Heat Waves,1.0,0.585597,-0.692398,-0.210741,0.631932
1,"The Kid LAROI, Justin Bieber",STAY (with Justin Bieber),1.0,-0.542116,0.750992,0.329911,-1.134620
2,Anitta,Envolver,3.0,0.923912,0.581892,0.353965,-0.187593
3,"Imagine Dragons, JID, Arcane, League of Legends",Enemy (with JID) - from the series Arcane Leag...,3.0,0.366688,0.865738,0.734636,-0.559574
4,GAYLE,abcdefu,1.0,0.147779,-0.601809,0.250493,-0.646610


In [11]:
new_specimen = {}
new_specimen['danceability'] = (0.593 - dance_avg)/dance_std
new_specimen['energy'] = (0.503 - e_avg)/e_std
new_specimen['loudness'] = (-6.725 - l_avg)/l_std
new_specimen['duration_ms'] = (137704.0 - duration_avg)/duration_std

print(knn(new_specimen, normalized_spotify, 1))

   artist_names                      track_name  peak_rank  danceability  \
46    Lil Nas X  MONTERO (Call Me By Your Name)        1.0     -0.528849   

      energy  loudness  duration_ms  distance to new  
46 -0.825263 -0.143923    -1.209326              0.0  


In [12]:
new_specimen = {}
new_specimen['danceability'] = (0.693 - dance_avg)/dance_std
new_specimen['energy'] = (0.599 - e_avg)/e_std
new_specimen['loudness'] = (-7.235 - l_avg)/l_std
new_specimen['duration_ms'] = (255000.0 - duration_avg)/duration_std

print(knn(new_specimen, normalized_spotify, 3))

              artist_names track_name  peak_rank  danceability    energy  \
255  The Weeknd, Daft Punk    Starboy        1.0      0.041641 -0.317962   
328   The Game, Kanye West       Eazy       35.0      0.287085 -0.535377   
20            Jaymes Young   Infinity       11.0     -0.011427  0.201416   

     loudness  duration_ms  distance to new  
255 -0.254650     0.479824         0.469881  
328 -0.243959     0.549668         0.508493  
20  -0.226395     0.612171         0.576771  


## Non-normalized Result
                                         artist_names  
240  Chris Jedi, Anuel AA, Chencho Corleone, Ñengo ...   
448                                     Kendrick Lamar   
615                                              Drake   

                             track_name   |  peak_rank  |  danceability   |energy  
240  La Llevo Al Cielo (Ft. Ñengo Flow)   |      53.0   |        0.795  | 0.845   
448                     United In Grief   |      11.0   |        0.529  | 0.845   
615                     Flight's Booked   |      49.0   |        0.593  | 0.410   

     loudness | duration_ms | distance to new  
240    -3.809  |   254920.0   |     80.073768  
448    -8.142  |   255378.0   |    378.001204  
615   -13.097  |   254407.0   |    593.029012  

## Normalized Result
              artist_names | track_name | peak_rank | danceability  |  energy  
255  The Weeknd, Daft Punk  |  Starboy  |      1.0  |    0.041641 | -0.317962   
328   The Game, Kanye West  |     Eazy  |     35.0  |    0.287085 | -0.535377   
20            Jaymes Young  | Infinity  |     11.0  |   -0.011427 | 0.201416   

     loudness | duration_ms | distance to new  
255 -0.254650 |    0.479824 |        0.469881  
328 -0.243959 |    0.549668 |        0.508493  
20  -0.226395 |    0.612171 |        0.576771  

## Analysis of Results
From the results of the non-normalized data, one can see that most of the values, except duration_ms, are all over the place. With this being the case, it is easy to say that duration_ms, with its high values, skewed the distance towards itself. Basically, it means that duration_ms became the most imporant deciding factor for the knn algorithm with the non-normalized data. Through normalization, one can see that the importance of the different predictors. duration_ms no longer holds all the sway for the song. Therefore, with the normalized data, all of the predictors are important deciding factors for the knn algorithm.

## Exercise #6

Do something cool. Using the movies dataset, get creative and show me something about the dataset. Use a text cell to explain what you did.

## Explanation
With the following function, one can input categories from a dataset. The function will sort the dataset by descending order of the categories and put the top k of peak_rank in a dictionary. It will also gather the mean rank of each category. My intention for this function is for someone to look at different categories and see if one may correlate better to peak_rank than others.

In [76]:
def analyze_k(categories, data, k):
  keys = categories
  len(keys)
  analysis = {}
  means = []
  data_copy = data.copy()
  for x in range(len(keys)):
    data_copy.sort_values([keys[x]], ascending=False, inplace=True)
    analysis[keys[x]] = data_copy['peak_rank'][0:k]
    means.append(data_copy['peak_rank'][0:k].mean())
    
  
  analysis['mean_rank'] = means
  return analysis

analyze_k(['danceability', 'energy'], spotify_copy, 5)


{'danceability': 622    105.0
 360     89.0
 44      45.0
 324    140.0
 596     16.0
 Name: peak_rank, dtype: float64, 'energy': 601    163.0
 434     21.0
 632     62.0
 118     55.0
 403    114.0
 Name: peak_rank, dtype: float64, 'mean_rank': [79.0, 83.0]}